In [1]:
import pandas as pd
import os
from glob import glob

base_path = "../../data/raw/bronze/teams/"

all_teams = []
all_matchs = []
for team_folder in os.listdir(base_path):
    team_path = os.path.join(base_path, team_folder)
    
    csv_files = glob(os.path.join(team_path, "*.csv"))
    
    for csv_file in csv_files:
        df = pd.read_csv(csv_file)
        if "players" in csv_file.lower() :

            mask = df.apply(lambda row: row.astype(str).str.contains("Playing Time|Performance|Opponent Total|Squad Total").any(), axis=1)
            df = df[~mask]
            
            df = df[~df["Player"].isin(["Player", "Ast"])]
            
            df["Team"] = team_folder
            
            df = df.reset_index(drop=True)
            
            all_teams.append(df)

        elif "matchs" in csv_file.lower():
            hide=df.apply(lambda row : row.astype(str).str.contains("Date|COMP|Round").any(), axis=1)
            df=df[~hide]
            df = df[~df["Date"].isin(["Date", "COMP"])]
            df["Team"] = team_folder
            df = df.reset_index(drop=True)
            all_matchs.append(df)

final_players_df = pd.concat(all_teams, ignore_index=True)
final_matchs_df=pd.concat(all_matchs, ignore_index=True)
output_players_file = "../../data/processed/all_players.csv"
output_matchs_file = "../../data/processed/all_matchs.csv"

os.makedirs(os.path.dirname(output_players_file), exist_ok=True)
os.makedirs(os.path.dirname(output_matchs_file ), exist_ok=True)

final_players_df.to_csv(output_players_file, index=False)


print("Tous les fichiers ont été fusionnés et nettoyés !")
print(final_players_df.shape)
print(final_players_df.head())
print(final_matchs_df.shape)
print(final_matchs_df.head())



Tous les fichiers ont été fusionnés et nettoyés !
(702, 17)
             Player   Nation    Pos Age  MP Starts   Min   90s Gls Ast G+A  \
0        David Raya   es ESP     GK  28  38     38  3420  38.0   0   0   0   
1    William Saliba   fr FRA     DF  23  35     35  3039  33.8   2   0   2   
2       Declan Rice  eng ENG     MF  25  35     33  2825  31.4   4   7  11   
3     Thomas Partey   gh GHA  MF,DF  31  35     31  2797  31.1   4   2   6   
4  Leandro Trossard   be BEL     FW  29  38     28  2546  28.3   8   7  15   

  G-PK PK PKatt CrdY CrdR     Team  
0    0  0     0    3    0  Arsenal  
1    2  0     0    2    1  Arsenal  
2    4  0     0    7    1  Arsenal  
3    4  0     0    4    0  Arsenal  
4    8  0     0    4    1  Arsenal  
(963, 18)
         Date            Comp         Round  Day Venue Result GF GA  \
0  2024-08-17  Premier League   Matchweek 1  Sat  Home      W  2  0   
1  2024-08-24  Premier League   Matchweek 2  Sat  Away      W  2  0   
2  2024-08-31  Premier Lea